In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
x_train = np.reshape(x_train, (-1,28,28,1))
x_test = np.reshape(x_test, (-1,28,28,1))
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [4]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
y_train.shape, y_test.shape

((60000, 10), (10000, 10))

In [6]:
## data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    validation_split = 0.25
)
datagen.fit(x_train)

In [10]:
## create model

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28,28,1), kernel_size=(5,5), filters=10, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=20, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy',
              metrics = ['accuracy']
             )

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 10)        260       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 10)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 20)        1820      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 20)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32064     
_________________________________________________________________
flatten_3 (Flatten)          (None, 64)               

In [11]:
train_generator = datagen.flow(x_train, y_train, batch_size = 128, subset = 'training')
val_generator = datagen.flow(x_train, y_train, batch_size = 128, subset = 'validation')

history = model.fit_generator( train_generator,
                               epochs = 5,
                               verbose = 1,
                               validation_data = val_generator
                               )


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
352/352 [==============================] - 28s 79ms/step - loss: 4.5784 - accuracy: 0.4156 - val_loss: 0.5376 - val_accuracy: 0.8403
Epoch 2/5
352/352 [==============================] - 28s 79ms/step - loss: 0.4205 - accuracy: 0.8717 - val_loss: 0.2950 - val_accuracy: 0.9095
Epoch 3/5
352/352 [==============================] - 28s 79ms/step - loss: 0.2504 - accuracy: 0.9248 - val_loss: 0.2065 - val_accuracy: 0.9355
Epoch 4/5
352/352 [==============================] - 28s 79ms/step - loss: 0.1799 - accuracy: 0.9442 - val_loss: 0.1629 - val_accuracy: 0.9500
Epoch 5/5
352/352 [==============================] - 28s 79ms/step - loss: 0.1442 - accuracy: 0.9550 - val_loss: 0.1435 - val_accuracy: 0.9571


In [12]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'test loss : {loss} / test accuracy : {accuracy}')

313/313 [==============================] - 2s 7ms/step - loss: 0.0608 - accuracy: 0.9808
test loss : 0.060754649341106415 / test accuracy : 0.9807999730110168
